# Optimising Gd concentration, TE and TR for SNR efficiency

Use this tool to find the optimum TR and Gd concentration for a range of TE values.

Use the first two cells to define the range of TE, TR and Gd parameters that you want to model, and your relaxometry data at different concentrations of Gd, then run all cells to see the result. To run a cell, select it and type ctrl + enter.

**Reference:** Barrett, RLC, Cash, D, Simmons, C, et al. Tissue Optimisation Strategies for High Quality Ex Vivo Diffusion Imaging. NMR in Biomedicine. 2022;e4866. https://doi.org/10.1002/nbm.4866

## 1. Enter data and parameters
Edit the values for `te`, `tr`, `gd`, `gd_data`, `t1_data` and `t2_data` in the cells below. 

First enter the range values of TE, TR and Gd concentration that you want to model (min, max):

In [ ]:
# TE and TR in ms 
te = (15, 25)
tr = (50, 100)

# Gd concentration in mM
gd = (0, 8)

Now enter your measurements of T1 and T2 at different concentrations of Gd:

In [ ]:
# Concentration of contrast agent in mM
gd_data = [0, 4, 8]
# Corresponding values of T1 and T2 in ms
t1_data = [1000, 80, 50]
t2_data = [45, 21, 14]

## 2. Run SNR efficiency model
Run the cell below to get the optimum TR and Gd concentration for the range of TE values defined above.

*Note:* If needed you can change the precision by altering the parameters `gd_step_size`, `te_step_size`, and `tr_step_size` in the code below. 

In [ ]:
# Import python packages

import numpy as np
from scipy.optimize import curve_fit

### First let's fit gd_data, t1_data and t2_data to the relaxivity vs contrast agent concentration function ###

# Define relaxivity vs Gd concentration function

def fun(gd, r_0, r_i):
    return r_0 + gd*r_i

# Curve fit for T1 data

# Define xdata and ydata
xdata = gd_data
# Convert T1 to R1 values
ydata = 1/np.array(t1_data)

# Curve fit for R1 parameters
r1_params, pcov = curve_fit(fun, xdata, ydata)

# Curve fit for T2 data

# Define xdata and ydata
xdata = gd_data
# Convert T2 to R2 values
ydata = 1/np.array(t2_data)

# Curve fit for R2 parameters
r2_params, pcov = curve_fit(fun, xdata, ydata)

# Convert curve parameters from relaxivity back to T1 and T2
t1_params = 1/r1_params
t2_params = 1/r2_params

### Now let's use these parameters to estimate T1 and T2 for the range of Gd concentration(s), TE and TR values given in gd, te, tr ###

# Rewrite relaxivity vs Gd concentration function in terms of T1 and T2
def fun_t(gd, t_0, t_i):
    return t_0 * t_i / (t_i + t_0 * gd)

# Define SNR efficiency function
def snreff_spin_echo(tr, te, t1, t2):
    '''SNR efficiency scaling factor for spin echo'''
    return (np.exp(-te / t2) * (1 - np.exp(-tr / t1) * (2 * np.exp(te / (2 * t1)) - 1))) / np.sqrt(tr)

# Define Gd concentrations
gd_step_size = 0.1
gd_vals = np.arange(gd[0], gd[1] + 1, gd_step_size)

# Define TE values concentrations
te_step_size = 1
te_vals = np.arange(te[0], te[1]+1, te_step_size)

# Define TR values concentrations
tr_step_size = 1
tr_vals = np.arange(tr[0], tr[1]+1, tr_step_size)

# Get T1 values
t_0, t_i = t1_params
t1_vals = fun_t(gd_vals, t_0, t_i)

# Get T2 values
t_0, t_i = t2_params
t2_vals = fun_t(gd_vals, t_0, t_i)

# Get SNR efficiency
snreff = np.zeros([len(te_vals), len(tr_vals), len(gd_vals)])
for i, TE in enumerate(te_vals):
    for j, (T1, T2) in enumerate(zip(t1_vals, t2_vals)):
        snreff[i, :, j] = snreff_spin_echo(tr_vals, TE, T1, T2)

# Find the maximum SNR efficiency accross Gd values
snreff_best_gd = np.max(snreff, axis=2)

# Find the Gd values that give maximum SNR efficiency
best_gd_index = np.argmax(snreff, axis=2).flatten()
best_gd = [gd_vals[i] for i in best_gd_index]
best_gd = np.reshape(best_gd, snreff_best_gd.shape)

# Print results
for i, TE in enumerate(te_vals):
    j = np.argmax(snreff_best_gd[i,:])
    TR = tr_vals[j]
    Gd = best_gd[i, j]
    print(f'For TE = {TE}, best TR = {TR} ms and best Gd = {Gd:.1f} mM.')